In [1]:
import numpy as np

from Solver import mesh_less, crank_nicolson_fd
from Utilz import F_v0, F_v0_x, diff, eval_F

import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import matplotlib.gridspec as gridspec

In [2]:
def mu(x, t, prms):
    T = prms[0]
    sigma = prms[1]
    a = prms[2]
    beta = prms[3]
    v0 = prms[4]
    
    beta_hat = beta*T
    v0_hat = beta*T - T*v0/a
    
    return v0_hat - beta_hat*x

In [3]:
def g(x, t, prms, max_k=6):
    T = prms[0]
    sigma = prms[1]
    a = prms[2]
    beta = prms[3]
    v0 = prms[4]
    
    kappa = (T*sigma**2)/(2*a**2)

    if t == 0:
        return 0
    elif np.isclose(mu(x, t, prms) - mu(0, 0, prms), 0):
        return 0
    else:
        return (mu(x, t, prms) - mu(0, 0, prms)) * F_v0_x(x, t, kappa, mu(0, 0, prms), max_k)

# Error Estimation:

In [16]:
sigma = 1
a = 1.2
T = 0.5
v0 = 1
beta = 0.4

M_ml_1 = 3
M_fd = M_ml_1

time_step = 10

max_k = 20

xx = np.linspace(0, 1, 21)
theta = .5

for T in [.2, .4, .6, 0.8, 1]:
    prms = [T, sigma, a, beta, v0]
    kappa = (T*sigma**2)/(2*a**2)
    
    ml_1 = mesh_less(kappa, prms, mu, g, theta=theta, M=M_ml_1, N=time_step, max_k=max_k, eps=0.2)
    ml_2 = mesh_less(kappa, prms, mu, g, theta=theta, M=M_ml_1, N=time_step, max_k=max_k, eps=0.4)
    ml_3 = mesh_less(kappa, prms, mu, g, theta=theta, M=M_ml_1, N=time_step, max_k=max_k, eps=0.6)
      
    fd = crank_nicolson_fd(kappa, prms, mu, g, theta=0.5, M=M_fd, N=time_step, max_k=max_k)
    print('Exact solution ...')
    fd_ex = fd_ex = np.load('_exact/exact_OU_{}.npy'.format(int(T*10)), allow_pickle=True)[()]
    
    exact = interp1d(fd_ex['X'], np.concatenate((np.array([0]), fd_ex['sol'], np.array([0]))), 'quadratic')
    fdp = interp1d(fd['X'], np.concatenate((np.array([0]), fd['sol'], np.array([0]))), 'quadratic')
    
    fd_err = np.sqrt(np.sum([np.abs(fdp(x) - exact(x))**2 for x in xx]))
    
    err_ml_1 = np.sqrt(np.sum([(eval_F(ml_1, x) - exact(x))**2 for x in xx]))
    err_ml_2 = np.sqrt(np.sum([(eval_F(ml_2, x) - exact(x))**2 for x in xx]))
    err_ml_3 = np.sqrt(np.sum([(eval_F(ml_3, x) - exact(x))**2 for x in xx]))
       
    print('T = {}, ML_1_ERR = {}, ML_2_ERR = {}, and ML_3_ERR = {}'.format(T, err_ml_1, err_ml_2, err_ml_3))
    print('FD_ERR = {},'.format(fd_err))
    print("\t\t\t\t....................................")

Exact solution ...
T = 0.2, ML_1_ERR = 0.006070984936560772, ML_2_ERR = 0.0058328121911313075, and ML_3_ERR = 0.0054549694692943495
FD_ERR = 0.007386689621270208,
				....................................
Exact solution ...
T = 0.4, ML_1_ERR = 0.005492601739403705, ML_2_ERR = 0.005625429107788856, and ML_3_ERR = 0.005828825066509726
FD_ERR = 0.012042592244386172,
				....................................
Exact solution ...
T = 0.6, ML_1_ERR = 0.006121578189097448, ML_2_ERR = 0.00645813772759656, and ML_3_ERR = 0.0069769773233267236
FD_ERR = 0.01472270452762912,
				....................................
Exact solution ...
T = 0.8, ML_1_ERR = 0.007804177067759537, ML_2_ERR = 0.008097925018761904, and ML_3_ERR = 0.008553799949326691
FD_ERR = 0.01622843883059446,
				....................................
Exact solution ...
T = 1, ML_1_ERR = 0.009374564962762707, ML_2_ERR = 0.009573830887140899, and ML_3_ERR = 0.00987653984241118
FD_ERR = 0.016852710718502955,
				..............................

In [332]:
sigma = 1
a = 1.2
T = 0.5
v0 = 1
beta = 0.4

time_step = 6

max_k = 20

xx = np.linspace(0, 1, 21)
theta = .5
T = .2

m1 = 5
m2 = 6

prms = [T, sigma, a, beta, v0]
kappa = (T*sigma**2)/(2*a**2)

ml_1 = mesh_less(kappa, prms, mu, g, theta=theta, M=m1, N=time_step, max_k=max_k, eps=.7)
ml_2 = mesh_less(kappa, prms, mu, g, theta=theta, M=m2, N=time_step, max_k=max_k, eps=.7)

fd_1 = crank_nicolson_fd(kappa, prms, mu, g, theta=0.5, M=m1, N=time_step, max_k=max_k)
fd_2 = crank_nicolson_fd(kappa, prms, mu, g, theta=0.5, M=m2, N=time_step, max_k=max_k)
print('Exact solution ...')
fd_ex = fd_ex = np.load('_exact/exact_OU_{}.npy'.format(int(T*10)), allow_pickle=True)[()]

exact = interp1d(fd_ex['X'], np.concatenate((np.array([0]), fd_ex['sol'], np.array([0]))), 'quadratic')
fdp_1 = interp1d(fd_1['X'], np.concatenate((np.array([0]), fd_1['sol'], np.array([0]))), 'quadratic')
fdp_2 = interp1d(fd_2['X'], np.concatenate((np.array([0]), fd_2['sol'], np.array([0]))), 'quadratic')

fd_err_1 = np.sqrt(np.sum([(fdp_1(x) - exact(x))**2 for x in xx]))
fd_err_2 = np.sqrt(np.sum([(fdp_2(x) - exact(x))**2 for x in xx]))

err_ml_1 = np.sqrt(np.sum([np.abs(eval_F(ml_1, x) - exact(x))**2 for x in xx]))
err_ml_2 = np.sqrt(np.sum([np.abs(eval_F(ml_2, x) - exact(x))**2 for x in xx]))

Exact solution ...


In [333]:
fd_err_1, fd_err_2, np.log2(fd_err_1/fd_err_2)

(0.0030348071834919777, 0.0021072221404189488, 0.526262448676134)

In [334]:
err_ml_1, err_ml_2, np.log2(err_ml_1/err_ml_2)

(0.0005302283355755449, 0.00024160593851740804, 1.133957854751777)

# Time Evaluation:

In [4]:
import timeit

In [13]:
sigma = 1
a = 1.2
T = 0.5
v0 = 1
beta = 0.4

M_ml_1 = 15
M_fd = M_ml_1

time_step = 10

max_k = 20

xx = np.linspace(0, 1, 21)
theta = .5    

start = timeit.default_timer()

for _ in range(10):
    for T in np.arange(0, 1+1/15, 1/15)[1:]:
        prms = [T, sigma, a, beta, v0]
        kappa = (T*sigma**2)/(2*a**2)

#         ml_1 = mesh_less(kappa, prms, mu, g, theta=theta, M=M_ml_1, N=time_step, max_k=max_k, eps=0.4)
    
        fd = crank_nicolson_fd(kappa, prms, mu, g, theta=0.5, M=M_fd, N=time_step, max_k=max_k)
        fdp = interp1d(fd['X'], np.concatenate((np.array([0]), fd['sol'], np.array([0]))), 'quadratic')


stop = timeit.default_timer()

print('Time: ', (stop - start)/10)

Time:  5.465309908299969
